In [19]:
import pandas as pd
import ast

In [33]:
df = pd.read_csv('Restaurantes_Yelp.csv')

In [34]:
df.columns

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'attributes', 'categories', 'hours'],
      dtype='object')

In [37]:
df['attributes'][1]

{'RestaurantsGoodForGroups': 'True',
 'RestaurantsTakeOut': 'True',
 'RestaurantsAttire': "'casual'",
 'NoiseLevel': "u'average'",
 'OutdoorSeating': 'True',
 'BikeParking': 'True',
 'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}",
 'RestaurantsPriceRange2': '2',
 'Caters': 'True',
 'GoodForKids': 'True',
 'RestaurantsReservations': 'True',
 'BusinessAcceptsCreditCards': 'True',
 'Alcohol': "'full_bar'",
 'GoodForMeal': "{'dessert': None, 'latenight': False, 'lunch': True, 'dinner': True, 'brunch': None, 'breakfast': False}",
 'RestaurantsTableService': 'True',
 'HappyHour': 'True',
 'WiFi': "u'free'",
 'DogsAllowed': 'False',
 'Ambience': "{u'divey': False, u'hipster': False, u'casual': True, u'touristy': False, u'trendy': False, u'intimate': False, u'romantic': None, u'classy': None, u'upscale': False}",
 'RestaurantsDelivery': 'True',
 'HasTV': 'True'}

In [36]:
df['attributes'] = df['attributes'].fillna('{}').apply(ast.literal_eval) #Completamos los valores faltantes en esta columna con '{}'


def convert_to_dict(val):
    # Transformamos en diccionarios reales los strings que parecen diccionarios
    if isinstance(val, str) and val.startswith("{") and val.endswith("}"):
        return ast.literal_eval(val)
    return val

df['attributes'] = df['attributes'].apply(convert_to_dict)

In [38]:
df_expanded = pd.json_normalize(df['attributes'].tolist()) #Combierte la columna en una lista de diccionarios y json_normalize pasa los valores a columnas
result_df = pd.concat([df.drop('attributes', axis=1), df_expanded], axis=1) #Combinamos el df original con el nuevo que tiene las columnas separadas

In [42]:
columns_to_drop = ['name', 'address', 'city', 'state', 'postal_code', 'latitude', 'longitude', 'stars', 'review_count', 'is_open', 'categories', 'hours'] #Eliminamos las columnas que no nos interesan
result_df = result_df.drop(columns=columns_to_drop)
result_df.columns

Index(['business_id', 'RestaurantsDelivery', 'OutdoorSeating',
       'BusinessAcceptsCreditCards', 'BusinessParking', 'BikeParking',
       'RestaurantsPriceRange2', 'RestaurantsTakeOut', 'ByAppointmentOnly',
       'WiFi', 'Alcohol', 'Caters', 'RestaurantsGoodForGroups',
       'RestaurantsAttire', 'NoiseLevel', 'GoodForKids',
       'RestaurantsReservations', 'GoodForMeal', 'RestaurantsTableService',
       'HappyHour', 'DogsAllowed', 'Ambience', 'HasTV',
       'BusinessAcceptsBitcoin', 'WheelchairAccessible', 'BYOB', 'DriveThru',
       'GoodForDancing', 'Music', 'CoatCheck', 'BestNights', 'Smoking',
       'BYOBCorkage', 'Corkage', 'Open24Hours', 'DietaryRestrictions',
       'AgesAllowed', 'RestaurantsCounterService', 'AcceptsInsurance'],
      dtype='object')

In [44]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15290 entries, 0 to 15289
Data columns (total 39 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   business_id                 15290 non-null  object
 1   RestaurantsDelivery         13928 non-null  object
 2   OutdoorSeating              12767 non-null  object
 3   BusinessAcceptsCreditCards  13156 non-null  object
 4   BusinessParking             13134 non-null  object
 5   BikeParking                 9726 non-null   object
 6   RestaurantsPriceRange2      13046 non-null  object
 7   RestaurantsTakeOut          14254 non-null  object
 8   ByAppointmentOnly           709 non-null    object
 9   WiFi                        10573 non-null  object
 10  Alcohol                     11596 non-null  object
 11  Caters                      9590 non-null   object
 12  RestaurantsGoodForGroups    12437 non-null  object
 13  RestaurantsAttire           11743 non-null  ob

In [45]:
result_df.to_csv('attributes.csv')